In [84]:
from datetime import datetime
start_time = datetime.now()

In [85]:
import pandas as pd
import numpy as np
import scipy.stats as sps
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
%matplotlib inline 

print('LOAD DURATION:',datetime.now() - start_time)

LOAD DURATION: 0:00:00.018165


In [86]:
%%time
data = pd.read_csv('NY property data.csv')
NY_data_orig = data.copy()

data.shape

CPU times: user 1.99 s, sys: 437 ms, total: 2.43 s
Wall time: 2.59 s


(1070994, 32)

In [87]:
data.dtypes

RECORD        int64
BBLE         object
BORO          int64
BLOCK         int64
LOT           int64
EASEMENT     object
OWNER        object
BLDGCL       object
TAXCLASS     object
LTFRONT       int64
LTDEPTH       int64
EXT          object
STORIES     float64
FULLVAL     float64
AVLAND      float64
AVTOT       float64
EXLAND      float64
EXTOT       float64
EXCD1       float64
STADDR       object
ZIP         float64
EXMPTCL      object
BLDFRONT      int64
BLDDEPTH      int64
AVLAND2     float64
AVTOT2      float64
EXLAND2     float64
EXTOT2      float64
EXCD2       float64
PERIOD       object
YEAR         object
VALTYPE      object
dtype: object

In [88]:
data.head()

,RECORD,BBLE,BORO,BLOCK,LOT,EASEMENT,OWNER,BLDGCL,TAXCLASS,LTFRONT,...,BLDFRONT,BLDDEPTH,AVLAND2,AVTOT2,EXLAND2,EXTOT2,EXCD2,PERIOD,YEAR,VALTYPE
0,1,1000010101,1,1,101,NaN,U S GOVT LAND & BLDGS,P7,4,500,...,0,0,3775500.0,8613000.0,3775500.0,8613000.0,NaN,FINAL,2010/11,AC-TR
1,2,1000010201,1,1,201,NaN,U S GOVT LAND & BLDGS,Z9,4,27,...,0,0,11111400.0,80690400.0,11111400.0,80690400.0,NaN,FINAL,2010/11,AC-TR
2,3,1000020001,1,2,1,NaN,DEPT OF GENERAL SERVI,Y7,4,709,...,709,564,32321790.0,40179510.0,32321790.0,40179510.0,NaN,FINAL,2010/11,AC-TR
3,4,1000020023,1,2,23,NaN,DEPARTMENT OF BUSINES,T2,4,793,...,85,551,13644000.0,15750000.0,13644000.0,15750000.0,NaN,FINAL,2010/11,AC-TR
4,5,1000030001,1,3,1,NaN,PARKS AND RECREATION,Q1,4,323,...,89,57,106348680.0,107758350.0,106348680.0,107758350.0,NaN,FINAL,2010/11,AC-TR


## Remove some benign properties that we aren't interested in 

In [89]:
# Looking at the most frequent owners
remove_list = data['OWNER'].value_counts().head(20).index.tolist()
remove_list

['PARKCHESTER PRESERVAT',
 'PARKS AND RECREATION',
 'DCAS',
 'HOUSING PRESERVATION',
 'CITY OF NEW YORK',
 'DEPT OF ENVIRONMENTAL',
 'BOARD OF EDUCATION',
 'NEW YORK CITY HOUSING',
 'CNY/NYCTA',
 'NYC HOUSING PARTNERSH',
 'YORKVILLE TOWERS ASSO',
 'DEPARTMENT OF BUSINES',
 'DEPT OF TRANSPORTATIO',
 'MTA/LIRR',
 'PARCKHESTER PRESERVAT',
 'MH RESIDENTIAL 1, LLC',
 '434 M LLC',
 'LINCOLN PLAZA ASSOCIA',
 'DEUTSCHE BANK NATIONA',
 '561 11TH AVENUE TMG L']

In [90]:
remove_list.remove('YORKVILLE TOWERS ASSO')
remove_list.remove('434 M LLC')
remove_list.remove('DEUTSCHE BANK NATIONA')
remove_list.remove('561 11TH AVENUE TMG L')
remove_list.append('UNITED STATES OF AMER')
remove_list.append('U S GOVERNMENT OWNRD')
remove_list.append('THE CITY OF NEW YORK')
remove_list.append('NYS URBAN DEVELOPMENT')
remove_list.append('NYS DEPT OF ENVIRONME')
remove_list.append('CULTURAL AFFAIRS')
remove_list.append('DEPT OF GENERAL SERVI')
remove_list.append('DEPT RE-CITY OF NY')
remove_list

['PARKCHESTER PRESERVAT',
 'PARKS AND RECREATION',
 'DCAS',
 'HOUSING PRESERVATION',
 'CITY OF NEW YORK',
 'DEPT OF ENVIRONMENTAL',
 'BOARD OF EDUCATION',
 'NEW YORK CITY HOUSING',
 'CNY/NYCTA',
 'NYC HOUSING PARTNERSH',
 'DEPARTMENT OF BUSINES',
 'DEPT OF TRANSPORTATIO',
 'MTA/LIRR',
 'PARCKHESTER PRESERVAT',
 'MH RESIDENTIAL 1, LLC',
 'LINCOLN PLAZA ASSOCIA',
 'UNITED STATES OF AMER',
 'U S GOVERNMENT OWNRD',
 'THE CITY OF NEW YORK',
 'NYS URBAN DEVELOPMENT',
 'NYS DEPT OF ENVIRONME',
 'CULTURAL AFFAIRS',
 'DEPT OF GENERAL SERVI',
 'DEPT RE-CITY OF NY']

In [91]:
property_data = data[-data['OWNER'].isin(remove_list)].reset_index(drop=True)
property_data.shape

(1046826, 32)

In [92]:
data.shape

(1070994, 32)

In [93]:
# Number of records removed
len(data) - len(property_data)

24168

In [94]:
numrecs = len(property_data)
ntop = 100

## Fill in missing ZIP

In [95]:
#Replacing missing zips. Replace NaN with 0, count them
missing_zips = np.where(pd.isnull(property_data['ZIP']))[0]
len(missing_zips)

21772

In [96]:
# Assume the data is already sorted by zip. If a zip is missing,
# and the before and after zips are the same, fill in the zip with that value

for i in range(len(missing_zips)):
    if(property_data.loc[missing_zips[i]+1,'ZIP'] == property_data.loc[missing_zips[i]-1,'ZIP']):
        property_data.loc[missing_zips[i],'ZIP'] = property_data.loc[missing_zips[i]-1,'ZIP']


In [97]:
# How many are still left to fill in?
missing_zips = np.where(pd.isnull(property_data['ZIP']))[0]
len(missing_zips)

10245

In [98]:
# For the remaining missing zips, just fill in with the previous record's zip.
for i in range(len(missing_zips)):
    property_data.loc[missing_zips[i],'ZIP'] = property_data.loc[missing_zips[i] - 1, 'ZIP']

In [99]:
missing_zips = np.where(pd.isnull(property_data['ZIP']))[0]
len(missing_zips)

0

## Calculate means for AVTOT, AVLAND, FULLVAL by taxclass, avoiding the records with zeros

In [100]:
property_data['FULLVAL'].replace('NaN',0)
temp = property_data[property_data['FULLVAL'] != 0]
mean_fullval = temp.groupby('TAXCLASS')['FULLVAL'].mean()
print(mean_fullval)

TAXCLASS
1     5.700059e+05
1A    3.352942e+05
1B    3.738399e+05
1C    7.615359e+05
1D    2.233614e+07
2     8.005832e+05
2A    8.639066e+05
2B    1.254525e+06
2C    7.723493e+05
3     8.510005e+04
4     2.772747e+06
Name: FULLVAL, dtype: float64


In [101]:
property_data['AVLAND'].replace('NaN',0)
temp_avland = property_data[property_data['AVLAND'] != 0]
mean_avland = temp_avland.groupby('TAXCLASS')['AVLAND'].mean()

In [102]:
property_data['AVTOT'].replace('NaN',0)
temp_avtot = property_data[property_data['AVTOT'] != 0]
mean_avtot = temp_avtot.groupby('TAXCLASS')['AVTOT'].mean()

## Substituting decent values for AVTOT, AVLAND, FULLVAL from averages by taxclass

In [103]:
%%time 
for index in mean_fullval.index:
    property_data.loc[(property_data['FULLVAL']==0) & (property_data['TAXCLASS']==index),'FULLVAL']=mean_fullval[index]
    property_data.loc[(property_data['AVLAND']==0) & (property_data['TAXCLASS']==index),'AVLAND']=mean_avland[index]
    property_data.loc[(property_data['AVTOT']==0) & (property_data['TAXCLASS']==index),'AVTOT']=mean_avtot[index]

CPU times: user 1.77 s, sys: 27.8 ms, total: 1.8 s
Wall time: 1.72 s


In [104]:
property_data.head().transpose()

,0,1,2,3,4
RECORD,1,2,9,10,11
BBLE,1000010101,1000010201,1000041001,1000041002,1000041003
BORO,1,1,1,1,1
BLOCK,1,1,4,4,4
LOT,101,201,1001,1002,1003
EASEMENT,NaN,NaN,NaN,NaN,NaN
OWNER,U S GOVT LAND & BLDGS,U S GOVT LAND & BLDGS,"TRZ HOLDINGS, LLC","TRZ HOLDINGS, LLC","TRZ HOLDINGS, LLC"
BLDGCL,P7,Z9,R5,R5,R5
TAXCLASS,4,4,4,4,4
LTFRONT,500,27,0,0,0


## Fill in the missing STORIES

In [105]:
temp = property_data[property_data['STORIES'].isnull()]
len(temp)

43968

In [106]:
temp['TAXCLASS'].value_counts()

1B    19055
4     17979
2      3434
3      2416
1       877
2C      138
2B       34
2A       30
1A        5
Name: TAXCLASS, dtype: int64

In [107]:
mean_stories = property_data.groupby('TAXCLASS')['STORIES'].mean()
mean_stories

TAXCLASS
1      2.111633
1A     1.656867
1B     4.000000
1C     3.052748
1D     1.068966
2     16.310549
2A     2.844032
2B     4.001635
2C     4.744977
3      1.000000
4      5.517257
Name: STORIES, dtype: float64

In [108]:
len(property_data[property_data['STORIES'] == 0])

0

In [109]:
property_data['STORIES'] = property_data['STORIES'].fillna(value = 0)
for index in mean_stories.index:
    property_data.loc[(property_data['STORIES']==0) & (property_data['TAXCLASS']==index),'STORIES'] = mean_stories[index]

In [110]:
property_data.head().transpose()

,0,1,2,3,4
RECORD,1,2,9,10,11
BBLE,1000010101,1000010201,1000041001,1000041002,1000041003
BORO,1,1,1,1,1
BLOCK,1,1,4,4,4
LOT,101,201,1001,1002,1003
EASEMENT,NaN,NaN,NaN,NaN,NaN
OWNER,U S GOVT LAND & BLDGS,U S GOVT LAND & BLDGS,"TRZ HOLDINGS, LLC","TRZ HOLDINGS, LLC","TRZ HOLDINGS, LLC"
BLDGCL,P7,Z9,R5,R5,R5
TAXCLASS,4,4,4,4,4
LTFRONT,500,27,0,0,0


## Fill in LTFRONT, LTDEPTH, BLDDEPTH, BLDFRONT with averages by TAXCLASS

In [111]:
# Because these 4 fields do not have NAs, we just need to replace 0s.
# We think zero and 1 are invalid values for these fields, so replace them with NA.
# Calculate groupwise average. Replace 0 and 1's by NAs so they are not counted in calculating mean.

property_data.loc[property_data['LTFRONT']==0,'LTFRONT']=np.nan
property_data.loc[property_data['LTDEPTH']==0,'LTDEPTH']=np.nan
property_data.loc[property_data['BLDFRONT']==0,'BLDFRONT']=np.nan
property_data.loc[property_data['BLDDEPTH']==0,'BLDDEPTH']=np.nan
property_data.loc[property_data['LTFRONT']==1,'LTFRONT']=np.nan
property_data.loc[property_data['LTDEPTH']==1,'LTDEPTH']=np.nan
property_data.loc[property_data['BLDFRONT']==1,'BLDFRONT']=np.nan
property_data.loc[property_data['BLDDEPTH']==1,'BLDDEPTH']=np.nan


In [112]:
# Calculate mean now (mean function ignores NAs but bot 0s hence we converted 0 to NA)
mean_LTFRONT = property_data.groupby(property_data['TAXCLASS'])['LTFRONT'].mean()
mean_LTDEPTH = property_data.groupby(property_data['TAXCLASS'])['LTDEPTH'].mean()
mean_BLDFRONT = property_data.groupby(property_data['TAXCLASS'])['BLDFRONT'].mean()
mean_BLDDEPTH = property_data.groupby(property_data['TAXCLASS'])['BLDDEPTH'].mean()

In [113]:
# Now impute values with the average for that record's TAXCLASS

for index in mean_LTFRONT.index:
    property_data.loc[(property_data['LTFRONT'].isnull()) & (property_data['TAXCLASS']==index),'LTFRONT']=mean_LTFRONT[index]
    property_data.loc[(property_data['LTDEPTH'].isnull()) & (property_data['TAXCLASS']==index),'LTDEPTH']=mean_LTDEPTH[index]
    property_data.loc[(property_data['BLDFRONT'].isnull()) & (property_data['TAXCLASS']==index),'BLDFRONT']=mean_BLDFRONT[index]
    property_data.loc[(property_data['BLDDEPTH'].isnull()) & (property_data['TAXCLASS']==index),'BLDDEPTH']=mean_BLDDEPTH[index]
    

In [114]:
mean_LTFRONT

TAXCLASS
1      30.939210
1A     51.130890
1B     46.316982
1C     27.250825
1D    338.275862
2     111.579150
2A     25.919459
2B     27.254800
2C     29.988866
3     154.473171
4      82.296384
Name: LTFRONT, dtype: float64

In [115]:
mean_LTDEPTH

TAXCLASS
1      99.279091
1A     90.095778
1B    103.993512
1C     96.778878
1D    497.827586
2     130.815876
2A     97.288381
2B     95.725639
2C     95.748523
3     274.124729
4     124.443774
Name: LTDEPTH, dtype: float64

In [116]:
mean_BLDFRONT

TAXCLASS
1     21.403255
1A    25.026348
1B    39.500000
1C    23.367589
1D    19.586207
2     91.495779
2A    22.985669
2B    26.033148
2C    27.887084
3     10.000000
4     62.082133
Name: BLDFRONT, dtype: float64

In [117]:
mean_BLDDEPTH

TAXCLASS
1      41.560577
1A     44.952387
1B     82.666667
1C     60.094862
1D     36.862069
2     104.282570
2A     62.435640
2B     66.521339
2C     68.657523
3      17.500000
4      86.296710
Name: BLDDEPTH, dtype: float64

In [118]:
data = property_data.copy()
data.shape

(1046826, 32)

In [119]:
data.dtypes

RECORD        int64
BBLE         object
BORO          int64
BLOCK         int64
LOT           int64
EASEMENT     object
OWNER        object
BLDGCL       object
TAXCLASS     object
LTFRONT     float64
LTDEPTH     float64
EXT          object
STORIES     float64
FULLVAL     float64
AVLAND      float64
AVTOT       float64
EXLAND      float64
EXTOT       float64
EXCD1       float64
STADDR       object
ZIP         float64
EXMPTCL      object
BLDFRONT    float64
BLDDEPTH    float64
AVLAND2     float64
AVTOT2      float64
EXLAND2     float64
EXTOT2      float64
EXCD2       float64
PERIOD       object
YEAR         object
VALTYPE      object
dtype: object

In [120]:
# convert ZIP to a string rather than a float
# We call the first three digits of the zip zip3

data['ZIP'] = data['ZIP'].astype(str)
data['zip3'] = data['ZIP'].str[:3]

In [121]:
data.count()

RECORD      1046826
BBLE        1046826
BORO        1046826
BLOCK       1046826
LOT         1046826
EASEMENT       2967
OWNER       1015081
BLDGCL      1046826
TAXCLASS    1046826
LTFRONT     1046826
LTDEPTH     1046826
EXT          353372
STORIES     1046826
FULLVAL     1046826
AVLAND      1046826
AVTOT       1046826
EXLAND      1046826
EXTOT       1046826
EXCD1        625308
STADDR      1046357
ZIP         1046826
EXMPTCL       10681
BLDFRONT    1046826
BLDDEPTH    1046826
AVLAND2      267417
AVTOT2       267423
EXLAND2       82371
EXTOT2       119371
EXCD2         92912
PERIOD      1046826
YEAR        1046826
VALTYPE     1046826
zip3        1046826
dtype: int64

In [122]:
cols = data.columns
print(cols)

Index(['RECORD', 'BBLE', 'BORO', 'BLOCK', 'LOT', 'EASEMENT', 'OWNER', 'BLDGCL',
       'TAXCLASS', 'LTFRONT', 'LTDEPTH', 'EXT', 'STORIES', 'FULLVAL', 'AVLAND',
       'AVTOT', 'EXLAND', 'EXTOT', 'EXCD1', 'STADDR', 'ZIP', 'EXMPTCL',
       'BLDFRONT', 'BLDDEPTH', 'AVLAND2', 'AVTOT2', 'EXLAND2', 'EXTOT2',
       'EXCD2', 'PERIOD', 'YEAR', 'VALTYPE', 'zip3'],
      dtype='object')


## Now build the variables

In [123]:
data['ltsize'] = data['LTFRONT']*data['LTDEPTH']
data['bldsize'] = data['BLDFRONT']*data['BLDDEPTH']
data['bldvol'] = data['bldsize']*data['STORIES']

In [124]:
data.head().transpose()

,0,1,2,3,4
RECORD,1,2,9,10,11
BBLE,1000010101,1000010201,1000041001,1000041002,1000041003
BORO,1,1,1,1,1
BLOCK,1,1,4,4,4
LOT,101,201,1001,1002,1003
EASEMENT,NaN,NaN,NaN,NaN,NaN
OWNER,U S GOVT LAND & BLDGS,U S GOVT LAND & BLDGS,"TRZ HOLDINGS, LLC","TRZ HOLDINGS, LLC","TRZ HOLDINGS, LLC"
BLDGCL,P7,Z9,R5,R5,R5
TAXCLASS,4,4,4,4,4
LTFRONT,500.0,27.0,82.296384,82.296384,82.296384


In [125]:
data['r1'] = data['FULLVAL'] / data['ltsize']
data['r2'] = data['FULLVAL'] / data['bldsize']
data['r3'] = data['FULLVAL'] / data['bldvol']
data['r4'] = data['AVLAND'] / data['ltsize']
data['r5'] = data['AVLAND'] / data['bldsize']
data['r6'] = data['AVLAND'] / data['bldvol']
data['r7'] = data['AVTOT'] / data['ltsize']
data['r8'] = data['AVTOT'] / data['bldsize']
data['r9'] = data['AVTOT'] / data['bldvol']

In [126]:
%%time

ninevars = ['r1','r2','r3','r4','r5','r6','r7','r8','r9']
zip5_mean = data.groupby('ZIP')[ninevars].mean()
zip3_mean = data.groupby('zip3')[ninevars].mean()
taxclass_mean = data.groupby('TAXCLASS')[ninevars].mean()
borough_mean = data.groupby('BORO')[ninevars].mean()

data = data.join(zip5_mean, on='ZIP', rsuffix='_zip5')
data = data.join(zip3_mean, on='zip3', rsuffix='_zip3')
data = data.join(taxclass_mean, on='TAXCLASS', rsuffix='_taxclass')
data = data.join(borough_mean, on='BORO', rsuffix='_boro')

rsuffix = ['_zip5','_zip3','_taxclass','_boro']

for var in ninevars:
    for r in rsuffix:
        data[str(var) + r] = data[var] / data[str(var) + r]


CPU times: user 1.36 s, sys: 450 ms, total: 1.81 s
Wall time: 1.76 s


In [127]:
data.head().transpose()

,0,1,2,3,4
RECORD,1,2,9,10,11
BBLE,1000010101,1000010201,1000041001,1000041002,1000041003
BORO,1,1,1,1,1
BLOCK,1,1,4,4,4
LOT,101,201,1001,1002,1003
...,...,...,...,...,...
r5_boro,16.606462,56.239136,2.499883,3.612808,3.802332
r6_boro,13.685555,46.347248,0.227331,0.328536,0.345771
r7_boro,0.198288,279.513149,1.866605,2.697598,5.015767
r8_boro,14.042014,127.165526,2.58843,3.740773,6.95539


In [128]:
data.columns

Index(['RECORD', 'BBLE', 'BORO', 'BLOCK', 'LOT', 'EASEMENT', 'OWNER', 'BLDGCL',
       'TAXCLASS', 'LTFRONT', 'LTDEPTH', 'EXT', 'STORIES', 'FULLVAL', 'AVLAND',
       'AVTOT', 'EXLAND', 'EXTOT', 'EXCD1', 'STADDR', 'ZIP', 'EXMPTCL',
       'BLDFRONT', 'BLDDEPTH', 'AVLAND2', 'AVTOT2', 'EXLAND2', 'EXTOT2',
       'EXCD2', 'PERIOD', 'YEAR', 'VALTYPE', 'zip3', 'ltsize', 'bldsize',
       'bldvol', 'r1', 'r2', 'r3', 'r4', 'r5', 'r6', 'r7', 'r8', 'r9',
       'r1_zip5', 'r2_zip5', 'r3_zip5', 'r4_zip5', 'r5_zip5', 'r6_zip5',
       'r7_zip5', 'r8_zip5', 'r9_zip5', 'r1_zip3', 'r2_zip3', 'r3_zip3',
       'r4_zip3', 'r5_zip3', 'r6_zip3', 'r7_zip3', 'r8_zip3', 'r9_zip3',
       'r1_taxclass', 'r2_taxclass', 'r3_taxclass', 'r4_taxclass',
       'r5_taxclass', 'r6_taxclass', 'r7_taxclass', 'r8_taxclass',
       'r9_taxclass', 'r1_boro', 'r2_boro', 'r3_boro', 'r4_boro', 'r5_boro',
       'r6_boro', 'r7_boro', 'r8_boro', 'r9_boro'],
      dtype='object')

In [129]:
save_record = data['RECORD']
save_record.head()

0     1
1     2
2     9
3    10
4    11
Name: RECORD, dtype: int64

In [130]:
dropcols = ['RECORD', 'BBLE', 'BORO', 'BLOCK', 'LOT', 'EASEMENT', 'OWNER', 'BLDGCL',
       'TAXCLASS', 'LTFRONT', 'LTDEPTH', 'EXT', 'STORIES', 'FULLVAL', 'AVLAND',
       'AVTOT', 'EXLAND', 'EXTOT', 'EXCD1', 'STADDR', 'ZIP', 'EXMPTCL',
       'BLDFRONT', 'BLDDEPTH', 'AVLAND2', 'AVTOT2', 'EXLAND2', 'EXTOT2',
       'EXCD2', 'PERIOD', 'YEAR', 'VALTYPE','zip3', 'ltsize', 'bldsize',
       'bldvol']

In [131]:
data = data.drop(columns=dropcols)
data.shape

(1046826, 45)

In [132]:
data.head().transpose()

,0,1,2,3,4
r1,40.917782,57678.882190,385.182796,556.662267,1035.027624
r2,3994.412456,36173.697854,736.308714,1064.105879,1978.540749
r3,723.985188,6556.463990,14.726174,21.282118,39.570815
r4,8.079350,4258.951518,62.110738,89.761892,94.470682
r5,788.709805,2671.030012,118.729804,171.587267,180.588507
r6,142.953243,484.122805,2.374596,3.431745,3.611770
r7,18.413002,25955.496986,173.332268,250.498069,465.762431
r8,1797.485605,16278.164034,331.338940,478.847739,890.343337
r9,325.793334,2950.408795,6.626779,9.576955,17.806867
r1_zip5,0.096029,135.364878,0.903974,1.306414,2.429076


In [133]:
# Calculate and write the basic statistics of all the variables 

stats = data.describe().transpose()
stats.to_excel('stats_on_vars.xlsx')

In [134]:
# data.to_csv('NY vars.csv', index = False)
data.describe()

,r1,r2,r3,r4,r5,r6,r7,r8,r9,r1_zip5,...,r9_taxclass,r1_boro,r2_boro,r3_boro,r4_boro,r5_boro,r6_boro,r7_boro,r8_boro,r9_boro
count,1.046826e+06,1.046826e+06,1.046826e+06,1.046826e+06,1.046826e+06,1.046826e+06,1.046826e+06,1.046826e+06,1.046826e+06,1.046826e+06,...,1.046826e+06,1.046826e+06,1.046826e+06,1.046826e+06,1.046826e+06,1.046826e+06,1.046826e+06,1.046826e+06,1.046826e+06,1.046826e+06
mean,2.135691e+02,5.364155e+02,2.440821e+02,1.046467e+01,2.281677e+01,1.004138e+01,2.683329e+01,5.046225e+01,1.907402e+01,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
std,5.325639e+02,1.027443e+03,4.988230e+02,6.997586e+01,3.785065e+02,1.783286e+02,2.038739e+02,9.322000e+02,3.270238e+02,2.163349e+00,...,5.627885e+00,2.365937e+00,2.002136e+00,2.542284e+00,6.308802e+00,1.714687e+01,1.604233e+01,7.276792e+00,2.193179e+01,1.572688e+01
min,3.905765e-04,7.466192e-04,8.983394e-05,8.333333e-06,2.096125e-04,1.038033e-05,8.333333e-06,3.062474e-04,6.049251e-05,2.228349e-06,...,4.896768e-06,2.317386e-06,1.323943e-06,3.128685e-07,9.337982e-07,4.413439e-06,9.937551e-07,3.875516e-07,8.187498e-06,2.777226e-06
25%,7.626263e+01,2.118164e+02,8.250000e+01,2.354081e+00,6.204263e+00,2.015872e+00,5.281000e+00,1.746754e+01,7.089338e+00,4.614558e-01,...,4.510961e-01,4.192226e-01,3.972038e-01,3.481865e-01,2.677718e-01,2.964940e-01,2.167336e-01,2.576625e-01,4.147249e-01,3.906492e-01
50%,1.528571e+02,5.000000e+02,2.390625e+02,4.596931e+00,1.498098e+01,7.146888e+00,8.579254e+00,2.697736e+01,1.267829e+01,9.159227e-01,...,8.398807e-01,8.383637e-01,9.485483e-01,9.177087e-01,6.692810e-01,7.748539e-01,6.965961e-01,5.143176e-01,6.964140e-01,6.729240e-01
75%,2.431818e+02,6.838235e+02,3.362314e+02,7.225445e+00,2.028646e+01,1.004363e+01,1.352700e+01,3.600000e+01,1.757576e+01,1.230053e+00,...,1.145319e+00,1.235402e+00,1.271407e+00,1.280152e+00,1.061934e+00,1.052615e+00,1.000643e+00,8.141981e-01,9.299754e-01,9.393327e-01
max,1.386373e+05,3.105516e+05,2.275000e+05,2.224980e+04,3.346364e+05,1.115455e+05,6.471185e+04,8.713622e+05,2.904541e+05,7.477557e+02,...,4.204082e+03,7.271440e+02,7.565810e+02,9.328696e+02,2.054622e+03,1.546654e+04,9.483223e+03,2.407601e+03,2.124127e+04,1.381933e+04
